### Setup
Install the Waymo Open Dataset Package and import the necessary packages.

In [ ]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/master
!pip3 install --upgrade pip

In [ ]:
!pip3 install waymo-open-dataset-tf-2-1-0==1.2.0

In [ ]:
import os
import tensorflow.compat.v1 as tf
import math
import numpy as np
import pandas as pd
import itertools

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

### Import Segment

Import Waymo Open segment data as .tfrecord file

In [ ]:
#FILENAME = 'Path to file segment.tfrecord'
FILENAME = '/content/validation_segment-12866817684252793621_480_000_500_000_with_camera_labels.tfrecord'

In [ ]:
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break

In [ ]:
(range_images, camera_projections,range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)

### Visualise Scene
Visualise the scene within the .tfrecord segment captured by cameras

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def show_camera_image(camera_image, camera_labels, layout, cmap=None):

  ax = plt.subplot(*layout)

  # Draw the camera labels.
  for camera_labels in frame.camera_labels:
    # Ignore camera labels that do not correspond to this camera.
    if camera_labels.name != camera_image.name:
      continue

  # Show the camera image.
  plt.imshow(tf.image.decode_jpeg(camera_image.image), cmap=cmap)
  plt.title(open_dataset.CameraName.Name.Name(camera_image.name))
  plt.grid(False)
  plt.axis('off')

plt.figure(figsize=(25, 20))

for index, image in enumerate(frame.images):
  show_camera_image(image, frame.camera_labels, [3, 3, index+1])

### Lidar Point Cloud Conversion

In [ ]:
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose)
points_ri2, cp_points_ri2 = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose,
    ri_index=1)

# 3d points in vehicle frame.
points_all = np.concatenate(points, axis=0)
points_all_ri2 = np.concatenate(points_ri2, axis=0)

# camera projection corresponding to each point.
cp_points_all = np.concatenate(cp_points, axis=0)
cp_points_all_ri2 = np.concatenate(cp_points_ri2, axis=0)

In [ ]:
images = sorted(frame.images, key=lambda i:i.name)
cp_points_all_concat = np.concatenate([cp_points_all, points_all], axis=-1)
cp_points_all_concat_tensor = tf.constant(cp_points_all_concat)

# The distance between lidar points and vehicle frame origin.
points_all_tensor = tf.norm(points_all, axis=-1, keepdims=True)
cp_points_all_tensor = tf.constant(cp_points_all, dtype=tf.int32)

##### Select LiDAR Point Cloud from scene

In [ ]:
mask = tf.equal(cp_points_all_tensor[..., 0], images[0].name)

"""Updating images[#] switches lidar point cloud
0 = Front
1 = Front Left
2 = Front Right"""

In [ ]:
cp_points_all_tensor = tf.cast(tf.gather_nd(
    cp_points_all_tensor, tf.where(mask)), dtype=tf.float32)
points_all_tensor = tf.gather_nd(points_all_tensor, tf.where(mask))

projected_points_all_from_raw_data = tf.concat(
    [cp_points_all_tensor[..., 1:3], points_all_tensor], axis=-1).numpy()

### LiDAR Point Cloud Visualisation

In [ ]:
def rgba(r):
  
  c = plt.get_cmap('jet')((r % 75.0) / 50.0)
  c = list(c)
  c[-1] = 0.8  #Adjust Transparency of Lidar Point Cloud (Alpha)
  return c

def plot_image(camera_image):

  plt.figure(figsize=(20, 12))
  plt.imshow(tf.image.decode_jpeg(camera_image.image))
  plt.grid("off")

def plot_points_on_image(projected_points, camera_image, rgba_func,
                         point_size=5.0):
  """Plots points on a camera image.

  Args:
    projected_points: [N, 3] numpy array. The inner dims are
      [camera_x, camera_y, range].
    camera_image: jpeg encoded camera image.
    rgba_func: a function that generates a color from a range value.
    point_size: the point size.

  """
  
  plot_image(camera_image)

  xs = []
  ys = []
  colors = []

  for point in projected_points:
    xs.append(point[0])  # width, col
    ys.append(point[1])  # height, row
    colors.append(rgba_func(point[2]))

  plt.scatter(xs, ys, c=colors, s=point_size, edgecolors="none")

In [ ]:
plot_points_on_image(projected_points_all_from_raw_data,
                     images[0], rgba, point_size=10.0)

"""Updating images[#] switches bg image
0 = Front
1 = Front Left
2 = Front Right"""

### Export Lidar Point Cloud without Image

In [ ]:
def plt_points_img(projected_points, rgba_func, point_size=5.0):

  xs = []
  ys = []
  colors = []

  for point in projected_points:
    xs.append(point[0])  # width, col
    ys.append(point[1])  # height, row
    colors.append(rgba_func(point[2]))

  plt.figure(figsize=(20, 8.5))
  plt.scatter(xs, ys, c=colors, s=point_size, edgecolors="none")

In [ ]:
# Call the function to plot image

plt_points_img(projected_points_all_from_raw_data, rgba, point_size=15.0)
#plt.grid("off")
plt.axis('off')
plt.gca().invert_yaxis()

### Export Image Frame for Monocular Depth Estimation

In [ ]:
def export_image(camera_image):

  plt.figure(figsize=(30, 20))
  plt.imshow(tf.image.decode_jpeg(camera_image.image))
  plt.axis('off')
  #plt.grid("off")


In [ ]:
export_image(images[0])
#plt.savefig('image5.png', bbox_inches='tight',pad_inches = 0)